In [ ]:
%load_ext autoreload
%autoreload 2
import os
import rawpy
import cv2
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import utils
import ISP as isp
import ISP.utils as utils
from denoise import LPfilter, DenoiseNet
from deblur import DeblurNet

In [ ]:
path = './data/real_scenes/'

filenames = [f for f in sorted(os.listdir(path)) if f.endswith('.RW2')]
img_n = 0 # Image number (0-3)
fn = filenames[img_n]
print(fn)

raw = rawpy.imread(path + fn)
pattern = raw.raw_pattern
byr = raw.raw_image_visible.copy()
utils.imshow(byr, titles='RAW Bayer Image')

In [ ]:
############# Preprocessing RAW image #############
blc = isp.black_level_correction(byr, black_level=raw.black_level_per_channel,
                                    white_level=raw.white_level, pattern=pattern,
                                    clip_range=[0, raw.white_level])

rgb = isp.demosaic_cv2(blc, pattern).astype('float32') / raw.white_level

# Display input image (postprocess for visualization)
utils.imshow(isp.postprocess(rgb, raw), titles='Input Pinhole Image')

In [ ]:
################## Denoise ##################
print('Denoising...')

LPFilter = LPfilter(device='cuda')
denoiseNet = DenoiseNet(device='cuda')

rgb_denoise = LPFilter(rgb)
rgb_denoise = denoiseNet(rgb_denoise)

utils.imshow(isp.postprocess(rgb_denoise, raw), titles='Denoised Pinhole Image')

In [ ]:
################## Deblur ##################
print('Debluring...')

deblurNet = DeblurNet(device='cuda')

rgb_deblur = deblurNet(rgb_denoise*255, None)

utils.imshow(isp.postprocess(rgb_deblur, raw), titles='Deblured Pinhole Image')